In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
from sklearn.metrics import accuracy_score,f1_score
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [5]:
!pip install tensorflow_text

ERROR: Operation cancelled by user


In [7]:
import tensorflow_hub as hub
import tensorflow_text as text

AttributeError: ignored

In [ ]:
!rm -rf ./logs/

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
data=pd.read_csv('gdrive/My Drive/oversampled_data.csv')
data=data.drop([data.columns[0]],axis=1)

In [ ]:
data

In [ ]:
data['medical_specialty'].value_counts()

In [ ]:
# data[['description','keywords']]=data[['description','keywords']].fillna("none")

Using only transcription as a feature for classification

In [ ]:
data_transcript=data[['transcription','medical_specialty']]

In [ ]:
data_transcript

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
X=data_transcript.iloc[:,0]
Y=data_transcript.iloc[:,-1]

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,shuffle=True,test_size=0.2)

In [ ]:
training_sentences=X_train.to_list()
testing_sentences=X_test.to_list()
training_labels=y_train.to_list()
testing_labels=y_test.to_list()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, categories='auto')
training_labels_final=encoder.fit_transform(np.array(training_labels).reshape(-1,1))
testing_labels_final=encoder.transform(np.array(testing_labels).reshape(-1,1))

In [ ]:
bert_preprocess = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3",trainable=False)
# bert_encoder = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2",trainable=False)
bert_encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/bert-en-uncased-l-10-h-128-a-2/versions/2",trainable=False)

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

In [ ]:
preprocessed_text = bert_preprocess(text_input)

In [ ]:
bert_outputs = bert_encoder(preprocessed_text)

In [ ]:
# bert_outputs = np.array(bert_outputs)
# bert_outputs.shape
bert_outputs

In [ ]:
# l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
# l = tf.keras.layers.Dense(200,activation = 'Sigmoid')
# l = tf.keras.layers.Dense(40, activation='softmax', name="output")(l)
# l_input = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
# l1 = tf.keras.layers.Dense(1000, activation='relu')(l_input)  # Specify the input tensor (l) for the Dense layer
# l2 = tf.keras.layers.Dense(200, activation='relu')(l1)
# l_output = tf.keras.layers.Dense(40, activation='softmax', name="output")(l2)



# l_input = tf.keras.layers.Dropout(0.1, name="dropout")(bert_outputs['pooled_output'])
# # l_input = tf.keras.layers.Reshape(target_shape=(768, 1))(l_input)
# l_input = tf.keras.layers.Reshape(target_shape=(128, 1))(l_input)
# print(l_input.shape)
# l1 = tf.keras.layers.Conv1D(500, 7,activation='relu')(l_input)
# l2 = tf.keras.layers.MaxPooling1D(5)(l1)

# l3 = tf.keras.layers.Conv1D(200, 15,activation='relu')(l2)
# l4 = tf.keras.layers.GlobalMaxPooling1D()(l3)

# l_output = tf.keras.layers.Dense(40, activation='softmax', name="output")(l4)



#REGULARIZED
# l_input = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
# l1 = tf.keras.layers.Conv1D(5000, 7,activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(l_input)
# l2 = tf.keras.layers.MaxPooling1D(5)(l1)

# l3 = tf.keras.layers.Conv1D(200, 15,activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(l2)
# l4 = tf.keras.layers.GlobalMaxPooling1D()(l3)

# l_output = tf.keras.layers.Dense(40, activation='softmax', name="output")(l4)





In [ ]:
# #CNN
# from keras.models import Sequential
# from keras.layers import Dropout,Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D
# from keras.layers import Embedding
# max_words = 20000
# max_length = 500

# model = Sequential()
# # l_input = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
# # l1 = tf.keras.layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))
# # l2 = tf.keras.layers.Dense(200, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))

# # l_output = tf.keras.layers.Dense(40, activation='softmax', name="output")
# model.add(Embedding(max_words, 32, input_length=max_length))
# model.add(Dropout(0.1, name="dropout"))
# model.add(Conv1D(32, 7, activation='relu'))
# model.add(MaxPooling1D(5))
# model.add(Conv1D(32, 7, activation='relu'))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(40, activation='sigmoid'))


# # model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# last_layer_output = model.layers[-1].output
# model = tf.keras.Model(inputs=[text_input], outputs = [l_output])
# model = tf.keras.Model(inputs=[text_input])


# model = tf.keras.Model(inputs=[bert_outputs], outputs = [l_output])

In [ ]:
# model.summary()

**Performance Metrics**

In [ ]:
# METRICS = [
#       tf.keras.metrics.BinaryAccuracy(name='accuracy'),
#       tf.keras.metrics.Precision(name='precision'),
#       tf.keras.metrics.F1Score(name='F1_score',average='micro')
# ]

# model.compile(optimizer='adam',
#  loss='categorical_crossentropy',
#  metrics=METRICS)

In [ ]:
# tf.config.run_functions_eagerly(True)

In [ ]:
# from sklearn.metrics import confusion_matrix
# import matplotlib.pyplot as plt
# import seaborn as sns


In [ ]:
# hist = model.fit(X_train, training_labels_final, epochs=1,validation_data=(X_test,testing_labels_final))

In [ ]:
# predictions = model.predict(X_test)
# predicted_labels = tf.argmax(predictions,axis=1)
# true_labels = tf.argmax(testing_labels_final, axis=1)
# cm = tf.math.confusion_matrix(true_labels, predicted_labels)


In [ ]:
# categories = ['Class 0', 'Class 1', 'Class 2']  # Replace with your class labels

# categories = data['medical_speciality']
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
# plt.xlabel('Predicted Labels')
# plt.ylabel('True Labels')
# plt.show()

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# %matplotlib inline
# sns.set()

# acc = hist.history['accuracy']
# val = hist.history['val_accuracy']
# epochs = range(1, len(acc) + 1)

# plt.plot(epochs, acc, '-', label='Training accuracy')
# plt.plot(epochs, val, ':', label='Validation accuracy')
# plt.title('Training and Validation Accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend(loc='lower right')
# plt.plot()

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the BERT preprocess module
bert_preprocess = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3",trainable=False)
# bert_encoder = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2",trainable=False)
bert_encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/bert-en-uncased-l-10-h-128-a-2/versions/2",trainable=False)

In [ ]:
# Define the input layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

# Preprocess the text using the BERT preprocess module
preprocessed_text = bert_preprocess(text_input)

# Pass the preprocessed text through the BERT encoder
bert_outputs = bert_encoder(preprocessed_text)

# Apply dropout and reshape the output
l_input = tf.keras.layers.Dropout(0.1, name="dropout")(bert_outputs['pooled_output'])
l_input = tf.keras.layers.Reshape(target_shape=(128,))(l_input)

# Convolutional layers
l1 = tf.keras.layers.Conv1D(500, 7, activation='relu')(l_input)
l2 = tf.keras.layers.MaxPooling1D(5)(l1)
l3 = tf.keras.layers.Conv1D(200, 15, activation='relu')(l2)
l4 = tf.keras.layers.GlobalMaxPooling1D()(l3)

# Output layer
l_output = tf.keras.layers.Dense(40, activation='softmax', name="output")(l4)

# Define the model
model = tf.keras.Model(inputs=[text_input], outputs=[l_output])

# Display model summary
model.summary()


In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.F1Score(name='F1_score',average='micro')
]

model.compile(optimizer='adam',
 loss='categorical_crossentropy',
 metrics=METRICS)

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
 hist = model.fit(X_train, training_labels_final, epochs=1,validation_data=(X_test,testing_labels_final))

In [ ]:
predictions = model.predict(X_test)
predicted_labels = tf.argmax(predictions,axis=1)
true_labels = tf.argmax(testing_labels_final, axis=1)
cm = tf.math.confusion_matrix(true_labels, predicted_labels)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()

acc = hist.history['accuracy']
val = hist.history['val_accuracy']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, '-', label='Training accuracy')
plt.plot(epochs, val, ':', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.plot()